In [1]:
import requests
from bs4 import BeautifulSoup
import time
import re
import wikipediaapi
from tqdm import tqdm
import os

https://en.wikipedia.org/wiki/Wikipedia:Vital_articles/Level/4/People <br>
https://en.wikipedia.org/wiki/Wikipedia:Vital_articles/Level/4/History <br>
https://en.wikipedia.org/wiki/Wikipedia:Vital_articles/Level/4/Geography <br>
https://en.wikipedia.org/wiki/Wikipedia:Vital_articles/Level/4/Arts <br>
https://en.wikipedia.org/wiki/Wikipedia:Vital_articles/Level/4/Philosophy_and_religion <br>
https://en.wikipedia.org/wiki/Wikipedia:Vital_articles/Level/4/Everyday_life <br>
https://en.wikipedia.org/wiki/Wikipedia:Vital_articles/Level/4/Society_and_social_sciences <br>
https://en.wikipedia.org/wiki/Wikipedia:Vital_articles/Level/4/Biology_and_health_sciences <br>
https://en.wikipedia.org/wiki/Wikipedia:Vital_articles/Level/4/Physical_sciences <br>
https://en.wikipedia.org/wiki/Wikipedia:Vital_articles/Level/4/Technology <br>
https://en.wikipedia.org/wiki/Wikipedia:Vital_articles/Level/4/Mathematics <br>

In [2]:
url = "https://en.wikipedia.org/wiki/Wikipedia:Vital_articles/Level/4/Mathematics"

In [3]:
path = "dataset/" + url.split("/")[-1]

In [4]:
os.makedirs(path, exist_ok=True)

In [5]:
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

tags = soup.find_all(lambda tag: tag.name == 'a' and 'href' in tag.attrs and 'title' in tag.attrs and len(tag.attrs) == 2)
article_names = [ tag.text for tag in tags]

In [6]:
pattern = '.*\/wiki.*'
success_count = 0
fail_count = 0
wiki_wiki = wikipediaapi.Wikipedia('en',timeout=10.0)

done_files = set()

for i, tag in enumerate( tqdm(tags) ):
    attributes = tag.attrs
    href = attributes["href"]
    title = attributes["title"]
    if re.match(pattern, href) and title not in done_files :
        #print(href + ":" + title)
        
        done_files.add(title)
        
        page_py = wiki_wiki.page(title)
        
        if page_py.exists() == False:
            #print("Failed for {}".format(title))
            fail_count += 1
            
        else:
            #print("Passed for {}".format(title))
            success_count += 1
            
            if "Category:Wikipedia" in title or "Wikipedia:" in title or "User:" in title or "Help:" in title:
                continue
            
            with open(path+"/{}.txt".format(title.replace('/','-')),'w') as file:
                file.write(page_py.text)
            
    if i%500 == 0 and i!=0:
        time.sleep(300)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 447/447 [02:09<00:00,  3.45it/s]


In [7]:
print(success_count)

373


In [8]:
print(fail_count)

10
